In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn import preprocessing

In [5]:
data = pd.read_csv('1429_1.csv')

c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
def sent(n):
    if n==5 or n==4:
        return 'Postive'
    elif n==3:
        return 'Nuetral'
    elif n==2 or n==1:
        return 'Negetive'
    else:
        pass

In [7]:
data_cols=data[['reviews.text','reviews.rating']]
data_cols['sentiments']=data_cols['reviews.rating'].apply(sent)

c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
data_cols_pos=data_cols[(data_cols['sentiments']=='Postive')].sample(812)
data_cols_neu=data_cols[(data_cols['sentiments']=='Nuetral')].sample(812)
data_cols_neg=data_cols[(data_cols['sentiments']=='Negetive')]
data_2=pd.concat([data_cols_pos,data_cols_neu,data_cols_neg])
label_encoder = preprocessing.LabelEncoder()
data_2['sentiments_en']=label_encoder.fit_transform(data_2['sentiments'])

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_2['reviews.text'],data_2['sentiments_en'], stratify=data_2['sentiments_en'])

In [14]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [15]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [17]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [18]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [19]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [20]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
58/58 [==============================] - 452s 8s/step - loss: -0.0188 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 2/10
58/58 [==============================] - 516s 9s/step - loss: -0.0902 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 3/10
58/58 [==============================] - 452s 8s/step - loss: -0.1795 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 4/10
58/58 [==============================] - 430s 7s/step - loss: -0.2549 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 5/10
58/58 [==============================] - 401s 7s/step - loss: -0.3347 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 6/10
58/58 [==============================] - 381s 7s/step - loss: -0.3821 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 7/10
58/58 [==============================] - 425s 7s/step - loss: -0.4575 - accuracy: 0.3333 - precision: 0.6667 - recall: 1.0000
Epoch 8/10
58/58 [=======================

In [21]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

20/20 [==============================] - 152s 8s/step


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       203
           1       0.33      1.00      0.50       203
           2       0.00      0.00      0.00       203

    accuracy                           0.33       609
   macro avg       0.11      0.33      0.17       609
weighted avg       0.11      0.33      0.17       609



c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\saifalikhan\OneDrive - OPTIMAL STRATEGIX GROUP INC\Desktop\Monkey_learn\.monlvenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in la